In [1]:
import re
import string
import time
from tqdm import tqdm

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver


In [2]:
def rendering(url):
    driver = webdriver.Chrome('/usr/local/bin/chromedriver') # run ChromeDriver
    driver.get(url) # load the web page from the URL
    time.sleep(3) # wait for the web page to load
    render = driver.page_source # get the page source HTML
    driver.quit() # quit ChromeDriver
    return render # return the page source HTML
    

In [3]:
base_url = 'https://d23.com/'

end_urls = ['0-9'] + list(string.ascii_lowercase)


In [14]:
df = pd.DataFrame(columns=['title', 'd23_link'])

for end in tqdm(end_urls):
    search_url = base_url + f'disney-a-to-z/{end}/'
    
    page = rendering(search_url)

    soup = BeautifulSoup(page, 'html.parser')
    
    page_content = soup.find_all('div', class_='a-z-letter-wrapper')
    
    for content in page_content:
        for a in content.select('a'):
            if ('film' and ('(' or ')')) in a.text:
                df = df.append({'title': a.text, 'd23_link': a['href'].lstrip(base_url)}, ignore_index=True)
            

100%|██████████| 27/27 [04:41<00:00, 10.43s/it]


In [5]:
df

,title,d23_link
0,10 Things I Hate About You (film),a-to-z/10-things-i-hate-about-you-film/
1,101 Dalmatians (film),a-to-z/101-dalmatians-film/
2,101 Dalmatians II: Patch’s London Adventure (f...,a-to-z/101-dalmatians-ii-patchs-london-adventu...
3,"101 Problems of Hercules, The (film)",a-to-z/101-problems-of-hercules-the-film/
4,102 Dalmatians (film),a-to-z/102-dalmatians-film/
...,...,...
2109,Zenith (film),a-to-z/zenith-film/
2110,Zokkomon (film),a-to-z/zokkomon-film/
2111,Zootopia (film),a-to-z/zootopia-film/
2112,Zorro the Avenger (film),a-to-z/zorro-the-avenger-film/


In [ ]:
translate_dict = dict([(ord(x), ord(y)) for x,y in zip( u"‘’´“”–-", u"'''\"\"--")]) 


In [ ]:
def get_d23(i, row):
    
    search_url = base_url + row['d23_link']

    page = rendering(search_url)

    soup = BeautifulSoup(page, 'html.parser')

    page_content = soup.find_all('div', class_='entry-content')

    # get film summary
    summary = []
    for content in page_content:
        for p in (content.select('p')):
            summary.extend(p.stripped_strings)
    
    # use regex to join summary list, do *not* add extra spaces between stripped string groups
    summary = re.sub("( [,.!?;\)]|[\(] | ['] )", lambda m: m.group(1).strip(), " ".join(summary))
    # translate typographical quotes, dashes, etc. to ascii characters
    summary = summary.translate(translate_dict)
    # remove all non-ascii characters
    summary = re.sub(r"[^\x00-\x7F]+"," ", summary)
    return summary


In [ ]:
df['summary'] = ''


In [ ]:
count = 0
iters = 10

#while count < iters:
for i,row in tqdm(df.iterrows(), total=df.shape[0]):

    if row['summary'] == '':
        summary = get_d23(i,row)
    
        df.at[i,'summary'] = summary
        
    break
    #count += 1
        

In [ ]:
df